In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

In [8]:
sum(param.numel() for param in model.parameters())

7850

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.9764
Validation accuracy: 0.7077, validation loss: 1.6995
Epoch 2
Training loss: 1.5101
Validation accuracy: 0.7834, validation loss: 1.3490
Epoch 3
Training loss: 1.2380
Validation accuracy: 0.8058, validation loss: 1.1378
Epoch 4
Training loss: 1.0683
Validation accuracy: 0.8211, validation loss: 1.0007
Epoch 5
Training loss: 0.9545
Validation accuracy: 0.8292, validation loss: 0.9055
Epoch 6
Training loss: 0.8731
Validation accuracy: 0.8365, validation loss: 0.8356
Epoch 7
Training loss: 0.8120
Validation accuracy: 0.8401, validation loss: 0.7820
Epoch 8
Training loss: 0.7644
Validation accuracy: 0.8447, validation loss: 0.7396
Epoch 9
Training loss: 0.7261
Validation accuracy: 0.8496, validation loss: 0.7051
Epoch 10
Training loss: 0.6947
Validation accuracy: 0.8533, validation loss: 0.6764
Epoch 11
Training loss: 0.6682
Validation accuracy: 0.8561, validation loss: 0.6522
Epoch 12
Training loss: 0.6457
Validation accuracy: 0.8583, validation loss: 0.6313
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8795, device='cuda:0'), tensor(0.4750, device='cuda:0'))
Valid: (tensor(0.8798, device='cuda:0'), tensor(0.4723, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8907, device='cuda:0'), tensor(0.3947, device='cuda:0'))
Valid: (tensor(0.8916, device='cuda:0'), tensor(0.3935, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8907, device='cuda:0'), tensor(0.3947, device='cuda:0'))
Valid: (tensor(0.8916, device='cuda:0'), tensor(0.3935, device='cuda:0'))


## Epoch average

In [17]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [18]:
log_file = open("SGD-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch_avg")

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.9689
Validation accuracy: 0.7349, validation loss: 1.6909
Epoch 2
Training loss: 1.5030
Validation accuracy: 0.7851, validation loss: 1.3429
Epoch 3
Training loss: 1.2332
Validation accuracy: 0.8055, validation loss: 1.1338
Epoch 4
Training loss: 1.0655
Validation accuracy: 0.8192, validation loss: 0.9985
Epoch 5
Training loss: 0.9530
Validation accuracy: 0.8268, validation loss: 0.9045
Epoch 6
Training loss: 0.8726
Validation accuracy: 0.8322, validation loss: 0.8355
Epoch 7
Training loss: 0.8122
Validation accuracy: 0.8395, validation loss: 0.7824
Epoch 8
Training loss: 0.7650
Validation accuracy: 0.8446, validation loss: 0.7404
Epoch 9
Training loss: 0.7270
Validation accuracy: 0.8481, validation loss: 0.7062
Epoch 10
Training loss: 0.6957
Validation accuracy: 0.8510, validation loss: 0.6777
Epoch 11
Training loss: 0.6694
Validation accuracy: 0.8543, validation loss: 0.6537
Epoch 12
Training loss: 0.6470
Validation accuracy: 0.8565, validation loss: 0.6328
E

In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8780, device='cuda:0'), tensor(0.4760, device='cuda:0'))
Valid: (tensor(0.8792, device='cuda:0'), tensor(0.4736, device='cuda:0'))


In [21]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [22]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [23]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8855, device='cuda:0'), tensor(0.4292, device='cuda:0'))
Valid: (tensor(0.8861, device='cuda:0'), tensor(0.4275, device='cuda:0'))


In [24]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [25]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [26]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8856, device='cuda:0'), tensor(0.4288, device='cuda:0'))
Valid: (tensor(0.8862, device='cuda:0'), tensor(0.4271, device='cuda:0'))
